In [1]:
import pandas as pd
import os
import sys
import json
import numpy as np
import tensorflow as tf
import keras.layers as L
import keras.activations as A
import keras.optimizers as O
import keras.losses as Loss
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras import backend as KB
from keras.callbacks import Callback
import time
from fc_conv import FC_CONV

ROOTDIR=os.path.abspath("../../")
sys.path.append(ROOTDIR)

from Preprocessing.Preprocessor import preprocessor
from data_utils.keras_generator import Generator
from utils.losses import weighted_cross_entropy_loss,f1_score_metric
from utils.callbacks import get_tensorboard_callback,get_checkpoint_call_back
from utils.tools import construct_model_from_csv
from models.BaseModel import Base_Model



from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from fc_conv import FC_CONV


/home/ai-trainingmachine/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
label_encoder_dir=os.path.join(ROOTDIR,"Preprocessing")
val_data_dir=r'/home/ai-trainingmachine/Desktop/Usman/internship.ai/val_data'

In [3]:
val_files=os.listdir(val_data_dir)
array=pd.read_hdf(os.path.join(val_data_dir,val_files[0]))
P=preprocessor(label_encoder_dir)
array=P.process_batch(array)
action_col=array.columns=='action'
g_config={'labels':6,
        'weightage':[1,1,1,1,1,1],
        'data_directory':val_data_dir,
        'file_batch_size':119,
        'format_':'h5',
        'ex_batch_size':10000,
        'file_examples':5000,
        'max_queue_size':1,
        'Preprocessor':P,
        'static_weightage':False,
         'action_col':action_col}

G=Generator(config=g_config)


/home/ai-trainingmachine/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [4]:
# nn_config={'layer_shapes':[300,300,300,100,50],'Activation':'relu','Output':6,'Input_shape':472, 'model_type':'fc'}
# my_model=Model(nn_config) 
# f1=f1_score_metric(6)
# my_model.model.compile(loss=weighted_cross_entropy_loss, optimizer=optimizers.Adam(lr=5e-4),metrics=["acc",f1])

In [4]:
input_shape=[(155,3),(7,)]
num_outputs=6
optimizer=O.Adam(lr=0.0001)
design_config_files=["conv_net.csv","fc_net.csv","merged_net.csv"]
model=FC_CONV(input_shape=input_shape,num_outputs=num_outputs,
        optimizer=optimizer,arch_config_files=design_config_files)
model.construct_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 155, 3, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 155, 3, 64)   640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 155, 3, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 155, 3, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

/home/ai-trainingmachine/Desktop/Armughanisback/internship.ai-feature-FC_CONV_model/models/fc_conv/fc_conv.py:48: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`
  self.Model=Model(inputs=[conv_input,scalar_features],output=logits)


In [5]:
G.generate()

/home/ai-trainingmachine/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


File No: 0
File No: 1
File No: 2
File No: 3
File No: 4
File No: 5
File No: 6
File No: 7
File No: 8
File No: 9
File No: 10
File No: 11
File No: 12
File No: 13
File No: 14
File No: 15
File No: 16
File No: 17
File No: 18
File No: 19
File No: 20
File No: 21
File No: 22
File No: 23
File No: 24
File No: 25
File No: 26
File No: 27
File No: 28
File No: 29
File No: 30
File No: 31
File No: 32
File No: 33
File No: 34
File No: 35
File No: 36
File No: 37
File No: 38
File No: 39
File No: 40
File No: 41
File No: 42
File No: 43
File No: 44
File No: 45
File No: 46
File No: 47
File No: 48
File No: 49
File No: 50
File No: 51
File No: 52
File No: 53
File No: 54
File No: 55
File No: 56
File No: 57
File No: 58
File No: 59
File No: 60
File No: 61
File No: 62
File No: 63
File No: 64
File No: 65
File No: 66
File No: 67
File No: 68
File No: 69
File No: 70
File No: 71
File No: 72
File No: 73
File No: 74
File No: 75
File No: 76
File No: 77
File No: 78
File No: 79
File No: 80
File No: 81
File No: 82
File No: 83
Fi

In [6]:
model.Model.load_weights(r'/home/ai-trainingmachine/Desktop/Armughanisback/internship.ai-feature-FC_CONV_model/checkpoints/FC_CONV_chkpoint__04.hdf5')

In [14]:
def convert_data(data):
    x=data
    ob_net=(x[:,2:467]).reshape(-1,155,3)
    num_cols=x.shape[1]

    scalar_features_col_mask=np.arange(num_cols)
    scalar_features_col_mask=(scalar_features_col_mask<2 ) |(scalar_features_col_mask>=467 )
    scalar_features=x[:,scalar_features_col_mask]
    return ob_net,scalar_features

In [12]:
G[0][0]

array([[-0.8985783 , -0.45257503, -0.5225416 , ..., -0.99999624,
         0.6022154 ,  0.70314795],
       [-1.3366468 , -0.45257503,  0.06798026, ..., -0.99999624,
        -0.15361218,  0.6100561 ],
       [-1.3366468 , -0.45257503, -0.5225416 , ..., -0.99999624,
         0.6022154 , -1.6098897 ],
       ...,
       [-0.02200311, -0.45257503,  0.06798026, ..., -0.99999624,
         0.6022154 ,  1.665315  ],
       [-0.8985783 , -0.45257503, -0.5225416 , ..., -0.99999624,
        -0.9094398 ,  1.3348923 ],
       [-1.3366468 , -0.45257503, -0.5225416 , ..., -0.99999624,
        -0.9094398 , -0.46113282]], dtype=float32)

In [17]:
true_actions=[]
pred_actions=[]
print (len(G))
for i in range(len(G)):
    data,actions=G[i]
    print (data.shape)
    obnet,scalars=convert_data(data)
    true_actions.extend(np.argmax(actions,axis=1))
    pred_actions.extend(np.argmax(model.Model.predict([obnet,scalars]),axis=1))
    print (i)
confusion_matrix(true_actions,pred_actions)

59
(10000, 472)
0
(10000, 472)
1
(10000, 472)
2
(10000, 472)
3
(10000, 472)
4
(10000, 472)
5
(10000, 472)
6
(10000, 472)
7
(10000, 472)
8
(10000, 472)
9
(10000, 472)
10
(10000, 472)
11
(10000, 472)
12
(10000, 472)
13
(10000, 472)
14
(10000, 472)
15
(10000, 472)
16
(10000, 472)
17
(10000, 472)
18
(10000, 472)
19
(10000, 472)
20
(10000, 472)
21
(10000, 472)
22
(10000, 472)
23
(10000, 472)
24
(10000, 472)
25
(10000, 472)
26
(10000, 472)
27
(10000, 472)
28
(10000, 472)
29
(10000, 472)
30
(10000, 472)
31
(10000, 472)
32
(10000, 472)
33
(10000, 472)
34
(10000, 472)
35
(10000, 472)
36
(10000, 472)
37
(10000, 472)
38
(10000, 472)
39
(10000, 472)
40
(10000, 472)
41
(10000, 472)
42
(10000, 472)
43
(10000, 472)
44
(10000, 472)
45
(10000, 472)
46
(10000, 472)
47
(10000, 472)
48
(10000, 472)
49
(10000, 472)
50
(10000, 472)
51
(5000, 472)
52
(10000, 472)
53
(10000, 472)
54
(10000, 472)
55
(10000, 472)
56
(10000, 472)
57
(10000, 472)
58


array([[     0,   1151,      0,      0,      0,      0],
       [     2, 269124,      0,      0,      0,      0],
       [    21, 126011,      0,      0,      0,      0],
       [     3, 186491,      0,      0,      0,      0],
       [     0,   1189,      0,      0,      0,      0],
       [     0,   1008,      0,      0,      0,      0]])

In [18]:
print (np.mean(true_actions==np.array(pred_actions).astype(int)))

0.4600410256410256


In [19]:
print (f1_score(true_actions,pred_actions,average='micro'))
print (f1_score(true_actions,pred_actions,average='macro'))
# print (f1_score(true_actions,pred_actions,average='samples'))
print (f1_score(true_actions,pred_actions,average='weighted'))

0.4600410256410256
0.10503219763493737
0.28991687405846317


/home/ai-trainingmachine/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
np.unique(true_actions,return_counts=True)

(array([0, 1, 2, 3, 4, 5]),
 array([   478, 115307,  53240,  80008,    517,    450]))